In [3]:
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing.pool import Pool

from collections import OrderedDict
import sys
import os
print(os.getcwd())
sys.path.insert(0, os.getcwd()+'/...') # +'/..'
sys.path.insert(0, os.getcwd()+'/..') # +'/..'
sys.path.insert(0, os.getcwd()+'/') # +'/..'
from IPython.display import clear_output

/home/sheikh/Motius/Thesis/synch_signal_detection_dl/tools/sionna/examples/myTests


In [4]:
import tensorflow as tf
from sionna.fec.scrambling import Scrambler
from sionna.fec.scrambling import Descrambler


2023-01-30 20:38:15.019689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 20:38:15.155735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-30 20:38:15.155766: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-30 20:38:15.175865: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-30 20:38:15.674442: W tensorflow/stream_executor/platform/de

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer


In [38]:
class IEEE80211Interleaver(Layer):
    
    def __init__(self,
                NCBPS,
                axis=-1,
                inverse=False,
                dtype=tf.float32,
                **kwargs):
        
        self.NCBPS = NCBPS
        self._inverse = inverse
        self._axis = axis
        
        super().__init__(dtype=dtype, **kwargs)
    
    def _generate_perm(self):
        assert self.NCBPS in (48,96,192,288),\
        "currently ony supports NCBPS  = 48,96,192, and 288"
        if self.NCBPS == 48:
            w48 = [1, 17, 33, 2, 18, 34, 3, 19, 35, 4, 20, 36, 5, 21, 37, 6, 22, 38, 7, 23, 39, 8, 24, 40, 9, 25, 41, 10, 26, 42, 11, 27, 43, 12, 28, 44, 13, 29, 45, 14, 30, 46, 15, 31, 47, 16, 32, 48]
            k48 = [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48]
            p =  [(w48[j]-1) for j in range(0, 48)]
            pinv =  [(k48[j]-1) for j in range(0, 48)]
        elif self.NCBPS == 96:
            w96 = [1, 17, 33, 49, 65, 81, 2, 18, 34, 50, 66, 82, 3, 19, 35, 51, 67, 83, 4, 20, 36, 52, 68, 84, 5, 21, 37, 53, 69, 85, 6, 22, 38, 54, 70, 86, 7, 23, 39, 55, 71, 87, 8, 24, 40, 56, 72, 88, 9, 25, 41, 57, 73, 89, 10, 26, 42, 58, 74, 90, 11, 27, 43, 59, 75, 91, 12, 28, 44, 60, 76, 92, 13, 29, 45, 61, 77, 93, 14, 30, 46, 62, 78, 94, 15, 31, 47, 63, 79, 95, 16, 32, 48, 64, 80, 96]
            k96 = [1, 7, 13, 19, 25, 31, 37, 43, 49, 55, 61, 67, 73, 79, 85, 91, 2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62, 68, 74, 80, 86, 92, 3, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 75, 81, 87, 93, 4, 10, 16, 22, 28, 34, 40, 46, 52, 58, 64, 70, 76, 82, 88, 94, 5, 11, 17, 23, 29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96] 
            p = [(w96[j]-1) for j in range(0, 96)]
            pinv = [(k96[j]-1) for j in range(0, 96)]
        elif self.NCBPS == 192:
            w192 = [1, 17, 33, 49, 65, 81, 97, 113, 129, 145, 161, 177, 18, 2, 50, 34, 82, 66, 114, 98, 146, 130, 178, 162, 3, 19, 35, 51, 67, 83, 99, 115, 131, 147, 163, 179, 20, 4, 52, 36, 84, 68, 116, 100, 148, 132, 180, 164, 5, 21, 37, 53, 69, 85, 101, 117, 133, 149, 165, 181, 22, 6, 54, 38, 86, 70, 118, 102, 150, 134, 182, 166, 7, 23, 39, 55, 71, 87, 103, 119, 135, 151, 167, 183, 24, 8, 56, 40, 88, 72, 120, 104, 152, 136, 184, 168, 9, 25, 41, 57, 73, 89, 105, 121, 137, 153, 169, 185, 26, 10, 58, 42, 90, 74, 122, 106, 154, 138, 186, 170, 11, 27, 43, 59, 75, 91, 107, 123, 139, 155, 171, 187, 28, 12, 60, 44, 92, 76, 124, 108, 156, 140, 188, 172, 13, 29, 45, 61, 77, 93, 109, 125, 141, 157, 173, 189, 30, 14, 62, 46, 94, 78, 126, 110, 158, 142, 190, 174, 15, 31, 47, 63, 79, 95, 111, 127, 143, 159, 175, 191, 32, 16, 64, 48, 96, 80, 128, 112, 160, 144, 192, 176]
            k192 = [1, 14, 25, 38, 49, 62, 73, 86, 97, 110, 121, 134, 145, 158, 169, 182, 2, 13, 26, 37, 50, 61, 74, 85, 98, 109, 122, 133, 146, 157, 170, 181, 3, 16, 27, 40, 51, 64, 75, 88, 99, 112, 123, 136, 147, 160, 171, 184, 4, 15, 28, 39, 52, 63, 76, 87, 100, 111, 124, 135, 148, 159, 172, 183, 5, 18, 29, 42, 53, 66, 77, 90, 101, 114, 125, 138, 149, 162, 173, 186, 6, 17, 30, 41, 54, 65, 78, 89, 102, 113, 126, 137, 150, 161, 174, 185, 7, 20, 31, 44, 55, 68, 79, 92, 103, 116, 127, 140, 151, 164, 175, 188, 8, 19, 32, 43, 56, 67, 80, 91, 104, 115, 128, 139, 152, 163, 176, 187, 9, 22, 33, 46, 57, 70, 81, 94, 105, 118, 129, 142, 153, 166, 177, 190, 10, 21, 34, 45, 58, 69, 82, 93, 106, 117, 130, 141, 154, 165, 178, 189, 11, 24, 35, 48, 59, 72, 83, 96, 107, 120, 131, 144, 155, 168, 179, 192, 12, 23, 36, 47, 60, 71, 84, 95, 108, 119, 132, 143, 156, 167, 180, 191]
            p = [(w192[j]-1) for j in range(0, 192)]
            pinv = [(k192[j]-1) for j in range(0, 192)]
        elif self.NCBPS == 288:
            w288 = [1, 17, 33, 49, 65, 81, 97, 113, 129, 145, 161, 177, 193, 209, 225, 241, 257, 273, 18, 34, 2, 66, 82, 50, 114, 130, 98, 162, 178, 146, 210, 226, 194, 258, 274, 242, 35, 3, 19, 83, 51, 67, 131, 99, 115, 179, 147, 163, 227, 195, 211, 275, 243, 259, 4, 20, 36, 52, 68, 84, 100, 116, 132, 148, 164, 180, 196, 212, 228, 244, 260, 276, 21, 37, 5, 69, 85, 53, 117, 133, 101, 165, 181, 149, 213, 229, 197, 261, 277, 245, 38, 6, 22, 86, 54, 70, 134, 102, 118, 182, 150, 166, 230, 198, 214, 278, 246, 262, 7, 23, 39, 55, 71, 87, 103, 119, 135, 151, 167, 183, 199, 215, 231, 247, 263, 279, 24, 40, 8, 72, 88, 56, 120, 136, 104, 168, 184, 152, 216, 232, 200, 264, 280, 248, 41, 9, 25, 89, 57, 73, 137, 105, 121, 185, 153, 169, 233, 201, 217, 281, 249, 265, 10, 26, 42, 58, 74, 90, 106, 122, 138, 154, 170, 186, 202, 218, 234, 250, 266, 282, 27, 43, 11, 75, 91, 59, 123, 139, 107, 171, 187, 155, 219, 235, 203, 267, 283, 251, 44, 12, 28, 92, 60, 76, 140, 108, 124, 188, 156, 172, 236, 204, 220, 284, 252, 268, 13, 29, 45, 61, 77, 93, 109, 125, 141, 157, 173, 189, 205, 221, 237, 253, 269, 285, 30, 46, 14, 78, 94, 62, 126, 142, 110, 174, 190, 158, 222, 238, 206, 270, 286, 254, 47, 15, 31, 95, 63, 79, 143, 111, 127, 191, 159, 175, 239, 207, 223, 287, 255, 271, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256, 272, 288]
            k288 = [1, 21, 38, 55, 75, 92, 109, 129, 146, 163, 183, 200, 217, 237, 254, 271, 2, 19, 39, 56, 73, 93, 110, 127, 147, 164, 181, 201, 218, 235, 255, 272, 3, 20, 37, 57, 74, 91, 111, 128, 145, 165, 182, 199, 219, 236, 253, 273, 4, 24, 41, 58, 78, 95, 112, 132, 149, 166, 186, 203, 220, 240, 257, 274, 5, 22, 42, 59, 76, 96, 113, 130, 150, 167, 184, 204, 221, 238, 258, 275, 6, 23, 40, 60, 77, 94, 114, 131, 148, 168, 185, 202, 222, 239, 256, 276, 7, 27, 44, 61, 81, 98, 115, 135, 152, 169, 189, 206, 223, 243, 260, 277, 8, 25, 45, 62, 79, 99, 116, 133, 153, 170, 187, 207, 224, 241, 261, 278, 9, 26, 43, 63, 80, 97, 117, 134, 151, 171, 188, 205, 225, 242, 259, 279, 10, 30, 47, 64, 84, 101, 118, 138, 155, 172, 192, 209, 226, 246, 263, 280, 11, 28, 48, 65, 82, 102, 119, 136, 156, 173, 190, 210, 227, 244, 264, 281, 12, 29, 46, 66, 83, 100, 120, 137, 154, 174, 191, 208, 228, 245, 262, 282, 13, 33, 50, 67, 87, 104, 121, 141, 158, 175, 195, 212, 229, 249, 266, 283, 14, 31, 51, 68, 85, 105, 122, 139, 159, 176, 193, 213, 230, 247, 267, 284, 15, 32, 49, 69, 86, 103, 123, 140, 157, 177, 194, 211, 231, 248, 265, 285, 16, 36, 53, 70, 90, 107, 124, 144, 161, 178, 198, 215, 232, 252, 269, 286, 17, 34, 54, 71, 88, 108, 125, 142, 162, 179, 196, 216, 233, 250, 270, 287, 18, 35, 52, 72, 89, 106, 126, 143, 160, 180, 197, 214, 234, 251, 268, 288]
            p = [(w288[j]-1) for j in range(0, 288)]
            pinv = [(k288[j]-1) for j in range(0, 288)]

        return p, pinv
    
    def build(self, input_shape):
        assert self._axis < len(input_shape), "Axis does match input shape"
        # init rand sequences during build
        assert input_shape[self._axis] is not None, "Unknown shape at req. dim"
        p, pi = self._generate_perm()
        self._perm_seq = tf.convert_to_tensor(p)
        self._perm_seq_inv = tf.convert_to_tensor(pi)
        
    def call(self, inputs):    

        input_shape = inputs.shape

        # re-init if shape has changed, update perm_seq
        if inputs.shape[self._axis] != self._perm_seq.shape[0]:
            self.build(inputs.shape)

        if self._inverse:
            x = tf.gather(inputs, self._perm_seq_inv, axis=self._axis)
        else:
            x = tf.gather(inputs, self._perm_seq, axis=self._axis)

        x = tf.ensure_shape(x, input_shape)
        return x
        
        
        



In [66]:

w48 = [1, 17, 33, 2, 18, 34, 3, 19, 35, 4, 20, 36, 5, 21, 37, 6, 22, 38, 7, 23, 39, 8, 24, 40, 9, 25, 41, 10, 26, 42, 11, 27, 43, 12, 28, 44, 13, 29, 45, 14, 30, 46, 15, 31, 47, 16, 32, 48]
w96 = [1, 17, 33, 49, 65, 81, 2, 18, 34, 50, 66, 82, 3, 19, 35, 51, 67, 83, 4, 20, 36, 52, 68, 84, 5, 21, 37, 53, 69, 85, 6, 22, 38, 54, 70, 86, 7, 23, 39, 55, 71, 87, 8, 24, 40, 56, 72, 88, 9, 25, 41, 57, 73, 89, 10, 26, 42, 58, 74, 90, 11, 27, 43, 59, 75, 91, 12, 28, 44, 60, 76, 92, 13, 29, 45, 61, 77, 93, 14, 30, 46, 62, 78, 94, 15, 31, 47, 63, 79, 95, 16, 32, 48, 64, 80, 96]
w192 = [1, 17, 33, 49, 65, 81, 97, 113, 129, 145, 161, 177, 18, 2, 50, 34, 82, 66, 114, 98, 146, 130, 178, 162, 3, 19, 35, 51, 67, 83, 99, 115, 131, 147, 163, 179, 20, 4, 52, 36, 84, 68, 116, 100, 148, 132, 180, 164, 5, 21, 37, 53, 69, 85, 101, 117, 133, 149, 165, 181, 22, 6, 54, 38, 86, 70, 118, 102, 150, 134, 182, 166, 7, 23, 39, 55, 71, 87, 103, 119, 135, 151, 167, 183, 24, 8, 56, 40, 88, 72, 120, 104, 152, 136, 184, 168, 9, 25, 41, 57, 73, 89, 105, 121, 137, 153, 169, 185, 26, 10, 58, 42, 90, 74, 122, 106, 154, 138, 186, 170, 11, 27, 43, 59, 75, 91, 107, 123, 139, 155, 171, 187, 28, 12, 60, 44, 92, 76, 124, 108, 156, 140, 188, 172, 13, 29, 45, 61, 77, 93, 109, 125, 141, 157, 173, 189, 30, 14, 62, 46, 94, 78, 126, 110, 158, 142, 190, 174, 15, 31, 47, 63, 79, 95, 111, 127, 143, 159, 175, 191, 32, 16, 64, 48, 96, 80, 128, 112, 160, 144, 192, 176]
w288 = [1, 17, 33, 49, 65, 81, 97, 113, 129, 145, 161, 177, 193, 209, 225, 241, 257, 273, 18, 34, 2, 66, 82, 50, 114, 130, 98, 162, 178, 146, 210, 226, 194, 258, 274, 242, 35, 3, 19, 83, 51, 67, 131, 99, 115, 179, 147, 163, 227, 195, 211, 275, 243, 259, 4, 20, 36, 52, 68, 84, 100, 116, 132, 148, 164, 180, 196, 212, 228, 244, 260, 276, 21, 37, 5, 69, 85, 53, 117, 133, 101, 165, 181, 149, 213, 229, 197, 261, 277, 245, 38, 6, 22, 86, 54, 70, 134, 102, 118, 182, 150, 166, 230, 198, 214, 278, 246, 262, 7, 23, 39, 55, 71, 87, 103, 119, 135, 151, 167, 183, 199, 215, 231, 247, 263, 279, 24, 40, 8, 72, 88, 56, 120, 136, 104, 168, 184, 152, 216, 232, 200, 264, 280, 248, 41, 9, 25, 89, 57, 73, 137, 105, 121, 185, 153, 169, 233, 201, 217, 281, 249, 265, 10, 26, 42, 58, 74, 90, 106, 122, 138, 154, 170, 186, 202, 218, 234, 250, 266, 282, 27, 43, 11, 75, 91, 59, 123, 139, 107, 171, 187, 155, 219, 235, 203, 267, 283, 251, 44, 12, 28, 92, 60, 76, 140, 108, 124, 188, 156, 172, 236, 204, 220, 284, 252, 268, 13, 29, 45, 61, 77, 93, 109, 125, 141, 157, 173, 189, 205, 221, 237, 253, 269, 285, 30, 46, 14, 78, 94, 62, 126, 142, 110, 174, 190, 158, 222, 238, 206, 270, 286, 254, 47, 15, 31, 95, 63, 79, 143, 111, 127, 191, 159, 175, 239, 207, 223, 287, 255, 271, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256, 272, 288]

k48 = [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48]
k96 = [1, 7, 13, 19, 25, 31, 37, 43, 49, 55, 61, 67, 73, 79, 85, 91, 2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62, 68, 74, 80, 86, 92, 3, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 75, 81, 87, 93, 4, 10, 16, 22, 28, 34, 40, 46, 52, 58, 64, 70, 76, 82, 88, 94, 5, 11, 17, 23, 29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
k192 = [1, 14, 25, 38, 49, 62, 73, 86, 97, 110, 121, 134, 145, 158, 169, 182, 2, 13, 26, 37, 50, 61, 74, 85, 98, 109, 122, 133, 146, 157, 170, 181, 3, 16, 27, 40, 51, 64, 75, 88, 99, 112, 123, 136, 147, 160, 171, 184, 4, 15, 28, 39, 52, 63, 76, 87, 100, 111, 124, 135, 148, 159, 172, 183, 5, 18, 29, 42, 53, 66, 77, 90, 101, 114, 125, 138, 149, 162, 173, 186, 6, 17, 30, 41, 54, 65, 78, 89, 102, 113, 126, 137, 150, 161, 174, 185, 7, 20, 31, 44, 55, 68, 79, 92, 103, 116, 127, 140, 151, 164, 175, 188, 8, 19, 32, 43, 56, 67, 80, 91, 104, 115, 128, 139, 152, 163, 176, 187, 9, 22, 33, 46, 57, 70, 81, 94, 105, 118, 129, 142, 153, 166, 177, 190, 10, 21, 34, 45, 58, 69, 82, 93, 106, 117, 130, 141, 154, 165, 178, 189, 11, 24, 35, 48, 59, 72, 83, 96, 107, 120, 131, 144, 155, 168, 179, 192, 12, 23, 36, 47, 60, 71, 84, 95, 108, 119, 132, 143, 156, 167, 180, 191]
k288 = [1, 21, 38, 55, 75, 92, 109, 129, 146, 163, 183, 200, 217, 237, 254, 271, 2, 19, 39, 56, 73, 93, 110, 127, 147, 164, 181, 201, 218, 235, 255, 272, 3, 20, 37, 57, 74, 91, 111, 128, 145, 165, 182, 199, 219, 236, 253, 273, 4, 24, 41, 58, 78, 95, 112, 132, 149, 166, 186, 203, 220, 240, 257, 274, 5, 22, 42, 59, 76, 96, 113, 130, 150, 167, 184, 204, 221, 238, 258, 275, 6, 23, 40, 60, 77, 94, 114, 131, 148, 168, 185, 202, 222, 239, 256, 276, 7, 27, 44, 61, 81, 98, 115, 135, 152, 169, 189, 206, 223, 243, 260, 277, 8, 25, 45, 62, 79, 99, 116, 133, 153, 170, 187, 207, 224, 241, 261, 278, 9, 26, 43, 63, 80, 97, 117, 134, 151, 171, 188, 205, 225, 242, 259, 279, 10, 30, 47, 64, 84, 101, 118, 138, 155, 172, 192, 209, 226, 246, 263, 280, 11, 28, 48, 65, 82, 102, 119, 136, 156, 173, 190, 210, 227, 244, 264, 281, 12, 29, 46, 66, 83, 100, 120, 137, 154, 174, 191, 208, 228, 245, 262, 282, 13, 33, 50, 67, 87, 104, 121, 141, 158, 175, 195, 212, 229, 249, 266, 283, 14, 31, 51, 68, 85, 105, 122, 139, 159, 176, 193, 213, 230, 247, 267, 284, 15, 32, 49, 69, 86, 103, 123, 140, 157, 177, 194, 211, 231, 248, 265, 285, 16, 36, 53, 70, 90, 107, 124, 144, 161, 178, 198, 215, 232, 252, 269, 286, 17, 34, 54, 71, 88, 108, 125, 142, 162, 179, 196, 216, 233, 250, 270, 287, 18, 35, 52, 72, 89, 106, 126, 143, 160, 180, 197, 214, 234, 251, 268, 288]


def interleave_symbs(bits, NCBPS):
    bits_inter = np.zeros(NCBPS)    

    if NCBPS == 48:
        a =  [(w48[j]-1) for j in range(0, 48)]
    elif NCBPS == 96:
        a = [(w96[j]-1) for j in range(0, 96)]
    elif NCBPS == 192:
        a = [(w192[j]-1) for j in range(0, 192)]
    elif NCBPS == 288:
        a = [(w288[j]-1) for j in range(0, 288)]

    bits_inter  = bits[a]
    return bits_inter

def deinterleave_symbs(bits, NCBPS):
    bits_deinter = np.zeros(NCBPS)    

    if NCBPS == 48:
        a =  [(k48[j]-1) for j in range(0, 48)]
    elif NCBPS == 96:
        a = [(k96[j]-1) for j in range(0, 96)]
    elif NCBPS == 192:
        a = [(k192[j]-1) for j in range(0, 192)]
    elif NCBPS == 288:
        a = [(k288[j]-1) for j in range(0, 288)]

    bits_deinter  = bits[a]
    return bits_deinter

In [ ]:
interleave_symbs()

In [67]:
s = tf.random.uniform([48,], minval=0, maxval=2, dtype=tf.int32)
s = s.numpy()
s

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1], dtype=int32)

In [68]:
NCBPS = 48
out_int = interleave_symbs(s, NCBPS)
out_int

array([1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1], dtype=int32)

In [69]:
out_deint = deinterleave_symbs(out_int, NCBPS)
out_deint

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1], dtype=int32)

In [70]:
NCBPS = 48
s = tf.random.uniform([NCBPS,], minval=0, maxval=2, dtype=tf.int32)
s = s.numpy()
interleaver = IEEE80211Interleaver(NCBPS,
                axis=-1,
                inverse=False,
                dtype=tf.float32)
deinterleaver = IEEE80211Interleaver(NCBPS,
                axis=-1,
                inverse=True,
                dtype=tf.float32)



s

array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0], dtype=int32)

In [71]:
interleaved = interleaver(s)
interleaved

<tf.Tensor: shape=(48,), dtype=int32, numpy=
array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0], dtype=int32)>

In [72]:
deinterleaved = deinterleaver(interleaved)
deinterleaved

<tf.Tensor: shape=(48,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0], dtype=int32)>

In [73]:
all(deinterleaved.numpy() == s)

True